# Direct join between French Crop Usage (FCU) and TAXREF-LD

Join condition: FCU crop name = TAXREF-LD vernacular name

Context:
- only consider FCU leaves or their direct parents
- only consider FCU crops under "usage des plantes cultivées", ignore "multi-usage" which is too ambiguous

### Initializations

In [7]:
import sys
import json
import os
from string import Template
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, POST
from time import sleep
from math import isnan, nan

In [8]:
sys.path.append('../..')
from utils import *

In [9]:
#fcu_endpoint = "http://ontology.inrae.fr/frenchcropusage/sparql"
fcu_endpoint = "http://localhost:8080/sparql"
taxref_endpoint = "https://taxref.mnhn.fr/sparql"

___
# Get all varieties from FCU
FCU labels = vernacular names

In [10]:
query =  prefixes + '''
select distinct ?fcu_concept ?fcu_name ?fcu_name_type where {

    # Select only cultivated crops
    <http://ontology.inrae.fr/frenchcropusage/Usages_plantes_cultivees> skos:narrower+ ?fcu_concept.

    # Keep only leaves or their direct parent but not above
    FILTER (NOT EXISTS { ?fcu_concept skos:narrower/skos:narrower ?child. })

    # Get prefered and alternate labels and keep track of the type of label
    ?fcu_concept a skos:Concept.
    { ?fcu_concept skos:prefLabel ?lb. bind("pref" as ?fcu_name_type) }
    UNION
    { ?fcu_concept skos:altLabel  ?lb. bind("alt" as ?fcu_name_type) }

    bind(str(lcase(?lb)) as ?fcu_name)

} order by ?fcu_concept
'''

In [13]:
%time df_fcu = exec_sparql(fcu_endpoint, query)

Wall time: 318 ms


In [14]:
dataframe_preview(df_fcu, end=5)

== Number of lines: 1515
== Number of unique values:
fcu_concept       446
fcu_name         1481
fcu_name_type       2
dtype: int64


fcu_concept                 fcu_name  \
0  http://ontology.inrae.fr/frenchcropusage/Abric...               abricotier   
1  http://ontology.inrae.fr/frenchcropusage/Abric...                  abricot   
2  http://ontology.inrae.fr/frenchcropusage/Abric...          abricotier pays   
3  http://ontology.inrae.fr/frenchcropusage/Abric...             abricot pays   
4  http://ontology.inrae.fr/frenchcropusage/Abric...  abricotier des antilles   

  fcu_name_type  
0          pref  
1           alt  
2          pref  
3           alt  
4           alt

---
# Join FCU with TAXREF-LD
### FCU varieties names = TAXREF-LD vernacular names

The SPARQL endpoint of TAXREF-LD does not support passing all varietes at once in a VALUES clause (ends up with an "HTTP Error 502: Proxy Error").
To avoid this, we make a SPARQL query for each FCU variety. It takes more time but it completes.

In [15]:
queryTpl = Template(prefixes + '''
select distinct ("$fcu_concept" as ?fcu_concept) ("$fcu_name" as ?fcu_name) ("$fcu_name_type" as ?fcu_name_type) ?taxref_full_name ?taxon ?rank (?vn as ?taxref_vn)
from <http://taxref.mnhn.fr/lod/graph/classes/15.0>
from <http://taxref.mnhn.fr/lod/graph/vernacular/15.0>
from <http://taxref.mnhn.fr/lod/graph/concepts>
where {
    ?taxon
       a                      owl:Class;
       taxrefp:hasReferenceName [ rdfs:label ?taxref_full_name ];
       taxrefp:hasRank        ?rank;
       taxrefp:vernacularName ?vn.

    # To lowercase + remove language tag if any
    filter (str(lcase(?vn)) = "$fcu_name")
    
    # All ranks up to spcecies but not above
    filter (?rank in (
        taxrefrk:Species,  taxrefrk:SemiSpecies, taxrefrk:MicroSpecies, taxrefrk:SubSpecies, taxrefrk:Natio, 
        taxrefrk:Varietas, taxrefrk:SubVarietas, taxrefrk:Forma,        taxrefrk:SubForma,   taxrefrk:FormaSpecies,
        taxrefrk:Linea,    taxrefrk:Clone,       taxrefrk:Race,         taxrefrk:Cultivar,   taxrefrk:Morpha,
        taxrefrk:Abberatio ))
}
''')

In [16]:
# Set a max number of queries to submit. 0 = unlimited.
MAX_QUERIES = 0

# Result DataFrame
df_fcu_taxref = pd.DataFrame()

for idx, row in df_fcu.iterrows():
    query = queryTpl.substitute(
        fcu_concept = row['fcu_concept'].strip(), 
        fcu_name = row['fcu_name'].strip().lower(),
        fcu_name_type = row['fcu_name_type'].strip().lower())
    #print(query)
    
    print(f"---- Running query {idx + 1}/{len(df_fcu)} - name = {row['fcu_name']}")
    %time _df = exec_sparql(taxref_endpoint, query)
    print(f'Number of results: {_df.shape[0]}')
    df_fcu_taxref = df_fcu_taxref.append(_df)
    
    # Keep track of the names not matched with TAXREF
    if _df.shape[0] == 0:
        nomatch_row = {'fcu_concept': row['fcu_concept'].strip(), 'fcu_name': row['fcu_name'].strip().lower(), 'fcu_name_type': row['fcu_name_type'].strip().lower(), 'taxref_full_name': None, 'taxon': None, 'rank': None}
        df_fcu_taxref = df_fcu_taxref.append(nomatch_row, ignore_index=True)

    if MAX_QUERIES > 0 and (idx+1) >= MAX_QUERIES:
        break

---- Running query 1/1515 - name = abricotier
Wall time: 880 ms
Number of results: 0
---- Running query 2/1515 - name = abricot
Wall time: 640 ms
Number of results: 0
---- Running query 3/1515 - name = abricotier pays
Wall time: 630 ms
Number of results: 0
---- Running query 4/1515 - name = abricot pays
Wall time: 709 ms
Number of results: 0
---- Running query 5/1515 - name = abricotier des antilles
Wall time: 801 ms
Number of results: 0
---- Running query 6/1515 - name = mamey
Wall time: 644 ms
Number of results: 0
---- Running query 7/1515 - name = abricotier-pays
Wall time: 653 ms
Number of results: 0
---- Running query 8/1515 - name = actinidia
Wall time: 645 ms
Number of results: 0
---- Running query 9/1515 - name = groseille de chine
Wall time: 617 ms
Number of results: 0
---- Running query 10/1515 - name = kiwi
Wall time: 662 ms
Number of results: 0
---- Running query 11/1515 - name = yang tao
Wall time: 684 ms
Number of results: 0
---- Running query 12/1515 - name = actinidier


In [32]:
# Number of matches
df_fcu_taxref_matched = df_fcu_taxref.dropna().drop_duplicates()
dataframe_preview(df_fcu_taxref_matched)

== Number of lines: 396
== Number of unique values:
fcu_concept         161
fcu_name            185
fcu_name_type         2
taxref_full_name    369
taxon               369
rank                  4
taxref_vn           189
dtype: int64


fcu_concept         fcu_name  \
26    http://ontology.inrae.fr/frenchcropusage/Ananas           ananas   
29  http://ontology.inrae.fr/frenchcropusage/Anemones          anémone   
35  http://ontology.inrae.fr/frenchcropusage/Angel...        angélique   
41  http://ontology.inrae.fr/frenchcropusage/Arach...         arachide   
67  http://ontology.inrae.fr/frenchcropusage/Artic...        artichaut   
72  http://ontology.inrae.fr/frenchcropusage/Asperges  asperge blanche   
80   http://ontology.inrae.fr/frenchcropusage/Attiers   pomme cannelle   
84  http://ontology.inrae.fr/frenchcropusage/Auber...        aubergine   
85  http://ontology.inrae.fr/frenchcropusage/Avoca...        avocatier   
91   http://ontology.inrae.fr/frenchcropusage/Avoines  avoine cultivée   

   fcu_name_type                                   taxref_full_name  \
26          pref                    Ananas comosus (L.) Merr., 1917   
29          pref  Anemone hortensis nothosubsp. fulgens (J.Gay) ...   
35          pref                 Dicorynia guianensis Amshoff, 1939   
41          pref                          Arachis hypogaea L., 1753   
67          pref                        Cynara cardunculus L., 1753   
72           alt                           Asparagus albus L., 1753   
80           alt                           Annona squamosa L., 1753   
84          pref                         Solanum melongena L., 1753   
85          pref                       Persea americana Mill., 1768   
91           alt                Avena sativa subsp. sativa L., 1753   

                                     taxon  \
26  http://taxref.mnhn.fr/lod/taxon/447782   
29  http://taxref.mnhn.fr/lod/taxon/131396   
35  http://taxref.mnhn.fr/lod/taxon/733639   
41  http://taxref.mnhn.fr/lod/taxon/611649   
67   http://taxref.mnhn.fr/lod/taxon/93783   
72   http://taxref.mnhn.fr/lod/taxon/84265   
80  http://taxref.mnhn.fr/lod/taxon/446901   
84  http://taxref.mnhn.fr/lod/taxon/124075   
85  http://taxref.mnhn.fr/lod/taxon/447273   
91  http://taxref.mnhn.fr/lod/taxon/132016   

                                            rank        taxref_vn  
26     http://taxref.mnhn.fr/lod/taxrank/Species           Ananas  
29  http://taxref.mnhn.fr/lod/taxrank/SubSpecies          Anémone  
35     http://taxref.mnhn.fr/lod/taxrank/Species        angélique  
41     http://taxref.mnhn.fr/lod/taxrank/Species         Arachide  
67     http://taxref.mnhn.fr/lod/taxrank/Species        Artichaut  
72     http://taxref.mnhn.fr/lod/taxrank/Species  Asperge blanche  
80     http://taxref.mnhn.fr/lod/taxrank/Species   Pomme cannelle  
84     http://taxref.mnhn.fr/lod/taxrank/Species        Aubergine  
85     http://taxref.mnhn.fr/lod/taxrank/Species        Avocatier  
91  http://taxref.mnhn.fr/lod/taxrank/SubSpecies  Avoine cultivée

### Matches:
- FCU: 161 unique concepts, 185 unique labels
- TAXREF-LD: 369 unique taxa from 4 ranks

### Exports

In [34]:
df_fcu_taxref.drop_duplicates().to_excel("result_fcu_taxref.xlsx")

In [35]:
# Reshape the DataFrame for later merging with other methods
df_fcu_taxref_matched.drop(columns=['fcu_name_type', 'fcu_name', 'taxref_vn'], inplace=True)
df_fcu_taxref_matched.rename(columns={'taxref_full_name': 'taxref_ref_full_name'}, inplace=True)
df_fcu_taxref_matched.insert(1, 'method', 'direct')
df_fcu_taxref_matched.insert(2, 'geves_name', '')
df_fcu_taxref_matched.insert(3, 'eppo_scientific_name', '')

In [36]:
dataframe_preview(df_fcu_taxref_matched)
df_fcu_taxref_matched.to_excel("result_fcu_taxref_merge.xlsx", index=False)
df_fcu_taxref_matched.to_csv("result_fcu_taxref_merge.csv", index=False)

== Number of lines: 396
== Number of unique values:
fcu_concept             161
method                    1
geves_name                1
eppo_scientific_name      1
taxref_ref_full_name    369
taxon                   369
rank                      4
dtype: int64


fcu_concept  method geves_name  \
26    http://ontology.inrae.fr/frenchcropusage/Ananas  direct              
29  http://ontology.inrae.fr/frenchcropusage/Anemones  direct              
35  http://ontology.inrae.fr/frenchcropusage/Angel...  direct              
41  http://ontology.inrae.fr/frenchcropusage/Arach...  direct              
67  http://ontology.inrae.fr/frenchcropusage/Artic...  direct              
72  http://ontology.inrae.fr/frenchcropusage/Asperges  direct              
80   http://ontology.inrae.fr/frenchcropusage/Attiers  direct              
84  http://ontology.inrae.fr/frenchcropusage/Auber...  direct              
85  http://ontology.inrae.fr/frenchcropusage/Avoca...  direct              
91   http://ontology.inrae.fr/frenchcropusage/Avoines  direct              

   eppo_scientific_name                               taxref_ref_full_name  \
26                                         Ananas comosus (L.) Merr., 1917   
29                       Anemone hortensis nothosubsp. fulgens (J.Gay) ...   
35                                      Dicorynia guianensis Amshoff, 1939   
41                                               Arachis hypogaea L., 1753   
67                                             Cynara cardunculus L., 1753   
72                                                Asparagus albus L., 1753   
80                                                Annona squamosa L., 1753   
84                                              Solanum melongena L., 1753   
85                                            Persea americana Mill., 1768   
91                                     Avena sativa subsp. sativa L., 1753   

                                     taxon  \
26  http://taxref.mnhn.fr/lod/taxon/447782   
29  http://taxref.mnhn.fr/lod/taxon/131396   
35  http://taxref.mnhn.fr/lod/taxon/733639   
41  http://taxref.mnhn.fr/lod/taxon/611649   
67   http://taxref.mnhn.fr/lod/taxon/93783   
72   http://taxref.mnhn.fr/lod/taxon/84265   
80  http://taxref.mnhn.fr/lod/taxon/446901   
84  http://taxref.mnhn.fr/lod/taxon/124075   
85  http://taxref.mnhn.fr/lod/taxon/447273   
91  http://taxref.mnhn.fr/lod/taxon/132016   

                                            rank  
26     http://taxref.mnhn.fr/lod/taxrank/Species  
29  http://taxref.mnhn.fr/lod/taxrank/SubSpecies  
35     http://taxref.mnhn.fr/lod/taxrank/Species  
41     http://taxref.mnhn.fr/lod/taxrank/Species  
67     http://taxref.mnhn.fr/lod/taxrank/Species  
72     http://taxref.mnhn.fr/lod/taxrank/Species  
80     http://taxref.mnhn.fr/lod/taxrank/Species  
84     http://taxref.mnhn.fr/lod/taxrank/Species  
85     http://taxref.mnhn.fr/lod/taxrank/Species  
91  http://taxref.mnhn.fr/lod/taxrank/SubSpecies